# датасет должен быть или скачен или сделан с помощью ноутбука RTSD-R_MERGED
Объединенный датасет доступен по [ссылке](https://drive.google.com/drive/folders/1jmxG2zfi-Fs3m2KrMGmjD347aYiT8YFM?usp=sharing).

Положить в папку data содержимое так, чтобы были следующие пути:  
* \$(ROOT_DIR)/data/merged-rtsd/...
* \$(ROOT_DIR)/data/gt.csv

> *gt_Set_NaN.csv - содержит тот же датасет, но значения колонки Set обнулено*

gt - датафрейм содержащий:  
* имена файлов - поле filename
* класс знака - поле sign_class
* флаг присутствия знака при работе с датасетом - IsPresent. Предполагается, что вместо удаления записи, будет устанавливатся этот флаг, включающий/не влючающий знак в выборку
* в какой набор включен знак - поле Set $\in$ $\{train, valid, test\}$

~~\# !gdown --id '1eKNfEuNQadRW1H4NOoMw5sdnyHV14ze0'
\# !unzip rtsd-r3.zip
\# !rm -rf rtsd-r3.zip~~

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
from torch import nn
import seaborn as sns
import pandas as pd
import os
import pathlib
import shutil
import cv2
import PIL
from datetime import datetime

%cd adas_system/notebooks

IN_COLAB = False
USE_COLAB_GPU = False
try:
    import google.colab
    IN_COLAB = True
    USE_COLAB_GPU = True
    from google.colab import drive
    drive.mount('/content/drive')
    if not os.path.isfile('1_ClassifierResearch.ipynb'):
        !git clone --branch 9_SignDetector https://github.com/lsd-maddrive/adas_system.git

    !gdown --id 1-K3ee1NbMmx_0T5uwMesStmKnZO_6mWi
    %cd adas_system/notebooks
    !mkdir ../data/R_MERGED
    !unzip -q -o /content/R_MERGED.zip -d ./../data/

except:
    if IN_COLAB:
        print('[!]YOU ARE IN COLAB, BUT DIDNT MOUND A DRIVE. Model wont be synced[!]')

        if not os.path.isfile('1_ClassifierResearch.ipynb'):
            !git clone --branch 9_SignDetector https://github.com/lsd-maddrive/adas_system.git
            !gdown --id 1-K3ee1NbMmx_0T5uwMesStmKnZO_6mWi
            %cd adas_system/notebooks
            !mkdir ../data/R_MERGED
            !unzip -q -o /content/R_MERGED.zip -d ./../data/

        IN_COLAB = False

    else:
        pass

###
import nt_helper
from nt_helper.helper_utils import *
###

TEXT_COLOR = 'black'

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
%matplotlib inline
plt.rcParams["figure.figsize"] = (17,10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Init dirs, init main vars

In [ ]:
if not IN_COLAB:
    PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))
else:
    PROJECT_ROOT = pathlib.Path('..')
    
DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'

gt = pd.read_csv(DATA_DIR / 'RTDS_DATASET.csv')

# FIX ME
SIGN_TO_NUMBER = pd.read_csv(DATA_DIR / 'sign_to_number.csv', index_col=0).T.to_dict('records')[0]
NUMBER_TO_SIGN = pd.read_csv(DATA_DIR / 'number_to_sign.csv', index_col=0).T.to_dict('records')[0]

gt['filepath'] = gt['filepath'].apply(lambda x: DATA_DIR / x)
GT_SRC_LEN = len(gt.index)
display(gt)

_, ax = plt.subplots(nrows=3, ncols=1, figsize=(21, 8))
LABELS = ['train', 'valid', 'test']

for i in range(len(LABELS)):
    g = sns.countplot(x='SIGN', 
                      data=gt[gt['SET']==LABELS[i]],  
                      ax=ax[i], 
                      order=sorted(gt['SIGN'].value_counts().index.tolist())
                     )
    ax[i].tick_params(labelrotation=90)
    ax[i].set_title(LABELS[i])
    plt.tight_layout()

In [ ]:
TRAIN_FILES_SET = set(gt[gt['SET']== 'train']['filepath'].values)
print(len(TRAIN_FILES_SET))
VALID_FILES_SET = set(gt[gt['SET']== 'valid']['filepath'].values)
print(len(VALID_FILES_SET))

In [ ]:
gt[gt['SET']== 'train']['SIGN'].value_counts()

In [ ]:
set.intersection(TRAIN_FILES_SET, VALID_FILES_SET)

Создадим загрузчик

In [ ]:
from sklearn import preprocessing

LE_LOCATION = DATA_DIR / 'le.npy'
le = preprocessing.LabelEncoder()

if os.path.isfile(LE_LOCATION):
    le.classes_ = np.load(LE_LOCATION)
else:
    le.fit_transform(gt['SIGN'])
    np.save(LE_LOCATION, le.classes_)

gt['ENCODED_LABELS'] = le.transform(gt['SIGN'])
display(gt)

class SignDataset(torch.utils.data.Dataset):
    def __init__(self, df, set_label, img_size=64, transform=None, le=None):
        
        if isinstance(img_size, int):
            img_size = (img_size, img_size)
        

        self.img_size = img_size
        self.df = df[df['SET']==set_label]

    def __len__(self):
        return len(self.df.index)
    
    def __getitem__(self, index):
        
        label = self.df.iloc[index]['ENCODED_LABELS']
        path = self.df.iloc[index]['filepath']
        # print(self.df.iloc[index])
        img = cv2.imread(str(path))
        img = cv2.resize(img, self.img_size)
        img_tnsr = torch.Tensor.permute(torch.Tensor(img), [2, 0, 1]).div(255)
        # print('ENCODED LABEL:', le.transform([label])[0])
        return img_tnsr, label # random.randrange(0, 7) #0#le.transform([label])[0]

img_size = 64    
train_dataset = SignDataset(gt, 'train', img_size)
valid_dataset = SignDataset(gt, 'valid', img_size)
test_dataset = SignDataset(gt, 'test', img_size)

MODEL_CLASSES = len(set(gt['SIGN']))

if IN_COLAB or USE_COLAB_GPU:
    batch_size = 512
else:
    batch_size = 1

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=False)

valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=False)

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=False)

In [ ]:
img_t, encoded_label = train_dataset[6]
print('encoded:', encoded_label)

decoded_label = le.inverse_transform([encoded_label])[0]
print('-le transform:', decoded_label)
sign = NUMBER_TO_SIGN[decoded_label]
print('-sing:', sign)

img = torch.Tensor.permute(torch.Tensor(img_t), [1, 2, 0]).cpu().detach().numpy()
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm


def train_epoch(model, loader, loss_op, optim, device, it_limit=99000):
    
    # Таким образом переводим модель в режим обучения
    # В этом режиме вычисляются градиенты, нужные для обучения
    torch.enable_grad()
    model.train()
    model.to(device)
    
    accur = []
    loss_val = []
    pbar = tqdm(enumerate(loader),
                total=len(loader), 
                position=0,
                leave=False)
    
    for idx, (data, target) in pbar:
        
        if it_limit and idx > it_limit:
            break
            
        data = data.to(device)
        target = target.to(device)
        
        optim.zero_grad()
        pred = model(data)
        
        local_acc = evaluate_batch_accuracy(pred, target).cpu()
        accur.append(local_acc)
        
        # print(pred)
        loss = loss_op(pred, target)
        loss_val.append(loss.item())
        
        # Gradient descent
        
        loss.backward()
        optim.step()
        
        
        pbar.set_description("train epoch mean accuracy: %.4f last_acc: %.4f" % (torch.mean(torch.stack(accur, dim=0)), local_acc))
        
    # print('train:', accur)
    return torch.mean(torch.stack(accur, dim=0))

from sklearn.metrics import accuracy_score

def evaluate_batch_accuracy_old(y_pred, y_true):
    '''
    Оценка точности предсказания (accuracy)

    y_pred:
        батч сырых степеней уверенности, размер (N, K)
    y_true:
        вектор истинных значений, размер (N)
    '''
    y_pred = y_pred.detach().numpy()
    y_true = y_true.detach().numpy()
    # print(y_true)
    # print(y_pred)
    accuracy = 0
    for i in range(len(y_true)):
        index_max = max(range(len(y_pred[i, :])), key=y_pred[i].__getitem__)
        # print(index_max)
        if (index_max == y_true[i]):
            accuracy += 1
    accuracy /= len(y_pred)
    return accuracy

def evaluate_batch_accuracy(y_pred, y_true):
    # print('y_pred', y_pred)
    # print('y_true', y_true)
    # return torch.from_numpy(np.array([evaluate_batch_accuracy_old(y_pred.cpu(), y_true.cpu())]))

    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    # print('y_pred_softmax', y_pred_tags)
    
    correct_pred = (y_pred_tags == y_true).float()
    # print(correct_pred)
    # s
    acc = correct_pred.sum() / len(correct_pred)
    
    # acc = torch.round(acc)
    # print(acc.dtype)
    return acc

def valid_epoch(model, loader, device, it_limit=9999):
    accur = []
    
    #torch.no_grad()
    #model.eval()
    model.to(device)
    
    pbar = tqdm(enumerate(loader),
                total=len(loader),
                position=0,
                leave=False)
        
    for idx, (imgs_batch, labels_batch) in pbar:
        imgs_batch = imgs_batch.to(device)

        if it_limit and idx > it_limit:
            break
            
        labels_batch = labels_batch.to(device)
        # print(labels_batch)
        pred = model(imgs_batch)
        # print('-\n', pred)
        local_acc = evaluate_batch_accuracy(pred, labels_batch).cpu()
        accur.append(local_acc)
        
        pbar.set_description("valid epoch accuracy: %f" % torch.mean(torch.stack(accur, dim=0)))
    ## print('valid acc:', accur)
    return torch.mean(torch.stack(accur, dim=0))


In [ ]:
config = {
    'lr': 0.005,
    'epochs': 10,
    'it_limit': None
}

DEFAULT_MODEL_LOCATION = DATA_DIR / 'resnet18_classifier'

from torchvision import models
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, MODEL_CLASSES)

if os.path.isfile(DEFAULT_MODEL_LOCATION):
    print('[+] Model restored from', DEFAULT_MODEL_LOCATION)
    model.load_state_dict(torch.load(DEFAULT_MODEL_LOCATION))

loss_op = nn.CrossEntropyLoss().cuda()
optim = torch.optim.SGD(model.parameters(), lr=config['lr'])

model.to(device)

SHOULD_I_TRAIN = True
if SHOULD_I_TRAIN:
    pbar = tqdm(range(config['epochs']),
            total=config['epochs'],
            position=0,
            leave=True)
    
    for epoch in pbar:

        train_res = train_epoch(model, train_loader, loss_op, optim, device, config['it_limit']) # 
        print('t:', train_res)

        valid_res = valid_epoch(model, valid_loader, device, config['it_limit'])
        print('v:', valid_res)
        
        #test_res = valid_epoch(model, test_loader, device, config['it_limit'])
        #print('!test:', test_res)
        
        now = datetime.now()
        model_save_name = 'resnet18_classifier_{}_T_ACC{:.4f}_V_ACC{:.4f}'.format(now.strftime("%m.%d_%H.%M"),
                                                                      train_res,
                                                                      valid_res)    
        pbar.set_description("per epoch valid accuracy %f" % valid_res)
        
        torch.save(model.state_dict(), DATA_DIR / model_save_name)
        if IN_COLAB:
            shutil.copy2(model_save_name, '/content/drive/MyDrive/')

        torch.save(model.state_dict(), DATA_DIR / 'resnet18_classifier')
        if IN_COLAB:
            shutil.copy2(DATA_DIR / 'resnet18_classifier', '/content/gdrive/MyDrive/')

    

In [ ]:
torch.__version__

In [ ]:
model.fc

In [ ]:
np.argmax([-3.67795, -4.46502, -4.00991, -4.30823, -1.97269, -2.26844, -3.99708, -3.03939, -3.76454, -2.84363, -2.88698, -3.26370, -3.60900, -2.46001, -2.57443, -2.67174, -4.14144, -4.07914, -2.78208, -1.07245, -1.77695,  6.58318, -0.19589, -3.07037, -2.55007, -2.18623, -0.31675, -3.51333, -2.96916, -4.83923, -3.64467])

In [ ]:
le.transform([42])

In [ ]:
le.inverse_transform([12])

# TEST MODEL

In [ ]:
gt.iloc[70380  ]

In [ ]:
def getRandomFromDataset(gt: pd.DataFrame, label='test', img_size=64, id_=None):
    
    if isinstance(img_size, int):
        img_size = (img_size, img_size)
    
    random_instance = gt[gt['SET']==label].sample(1)

    if id_:
        random_instance = gt[gt['SET']==label].iloc[[id_], :]
    
    # print(random_instance)
    img_path = str(random_instance['filepath'].values[0])

    sign_class = random_instance['SIGN'].values[0]
    img = cv2.imread(img_path)
    
    img_model = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_model = cv2.resize(img, img_size)
    
    model_input = torch.Tensor.permute(torch.Tensor(img_model), [2, 0, 1]).div(255)[None, ...]
    
    return model_input, img, le.transform([sign_class])[0]

def translateNumber2Sign(le, encoded_label):
    return NUMBER_TO_SIGN[le.transform([encoded_label])[0]]
    

model.eval()
model.to(device)

model_input, img, encoded_label = getRandomFromDataset(new_mini_df, label='train')

# print('encoded_label:', encoded_label)
#decoded = le.inverse_transform([encoded_label])
print('decoded_label:', le.inverse_transform([encoded_label]))
print('sign', NUMBER_TO_SIGN[le.inverse_transform([encoded_label])[0]])

preds = model(model_input.to(device)).cpu().detach().numpy()
print(preds)
print('argmax', np.argmax(preds))
print('Predicted:',  NUMBER_TO_SIGN[np.argmax(preds)])

fig = plt.figure(figsize=(2,2))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
le.transform([22])

In [ ]:
gt.iloc[34290  ]

In [ ]:
le.classes_

In [ ]:
le.transform([24])

In [ ]:
SIGN_TO_NUMBER[4]

In [ ]:
# print(len(test_loader))
model.to('cpu')
iters = 1

for data, target in test_loader:
    
    data.to(device)
    
    print('target:', target)
    t = target.cpu().detach().numpy()
    # print('sign', NUMBER_TO_SIGN[le.inverse_transform([t[1]])[0]])

    preds = model(data)# .detach().numpy()
    
    acc = evaluate_batch_accuracy(preds, target)
    
    print('accuracy', acc)
    print(preds)
    _, argmaxes = torch.max(preds, dim=1)
    print(argmaxes)
    #print('argmax', np.argmax(preds))
    #print('Predicted:',  NUMBER_TO_SIGN[np.argmax(preds)])

    #img = torch.Tensor.permute(data[0], [1, 2, 0]).detach().numpy()
    #plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    break

In [ ]:
a = model(img_t[None, ...])
a.shape
np.argmax(a.cpu().detach().numpy())

In [ ]:
test_acc

In [ ]:
!7z a resnets resnet18_*

In [ ]:
%load_ext autoreload
%autoreload 2
###
import nt_helper
from nt_helper.helper_utils import *
###



img_t, label_e = test_dataset[3]
showTensorPicture(img_t, label=MODEL_CLASS_UNMAP[label_e])
print("PREDICTED SIGN:", MODEL_CLASS_UNMAP[label_e])

PICK RANDOM IMAGE FROM EACH SIGN CLASS for TRAIN

In [ ]:
gt_ = gt[gt["SET"]=='train']
SIGN_SET = set(gt_['SIGN'])

nrows, ncols = 6, 7
fig = plt.figure()

new_mini_df = pd.DataFrame(columns=gt_.columns)
# display(new_mini_df)

for idx, sign_class in enumerate(SIGN_SET):
    
    instance = gt_[gt_['SIGN'] == sign_class].sample(1)
    # display(instance)
    new_mini_df.loc[len(new_mini_df)] = instance.iloc[0]
    # print(new_mini_df)
    path = str(instance['filepath'].values[0])
    sign = instance['SIGN'].values[0]
    img = cv2.imread(path)
    ax = fig.add_subplot(nrows, ncols, idx+1)
    
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), aspect=1)
    ax.set_title(str(le.transform([sign_class])[0]) + ':' + str(sign_class) + ':' + str(NUMBER_TO_SIGN[sign_class]))
    
plt.tight_layout()

In [ ]:
display(new_mini_df[::4])

In [ ]:
class SignDataset(torch.utils.data.Dataset):
    def __init__(self, df, set_label, img_size=64, transform=None, le=None):
        
        if isinstance(img_size, int):
            img_size = (img_size, img_size)
        

        self.img_size = img_size
        self.df = df[df['SET']==set_label]

    def __len__(self):
        return len(self.df.index)
    
    def __getitem__(self, index):
        
        label = self.df.iloc[index]['ENCODED_LABELS']
        path = self.df.iloc[index]['filepath']
        # print(self.df.iloc[index])
        img = cv2.imread(str(path))
        img = cv2.resize(img, self.img_size)
        img_tnsr = torch.Tensor.permute(torch.Tensor(img), [2, 0, 1]).div(255)
        # print('ENCODED LABEL:', le.transform([label])[0])
        return img_tnsr, torch.tensor(label, dtype=torch.long) # random.randrange(0, 7) #0#le.transform([label])[0]

small_test_loader = SignDataset(new_mini_df, 'train', 64)
print('LOADER SIZE =', len(small_test_loader))

train_loader = torch.utils.data.DataLoader(
        small_test_loader,
        batch_size=1,
        pin_memory=True,
        shuffle=False)

In [ ]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

def train_epoch(model, loader, loss_op, optim, device, it_limit=99000):

    torch.enable_grad()
    model.train()
    model.to(device)
    
    accur = []
    loss_val = []
    pbar = tqdm(enumerate(loader),
                total=len(loader), 
                position=0,
                leave=False)
    
    for idx, (data, target) in pbar:
        
        if it_limit and idx > it_limit:
            break
            
        data = data.to(device)
        target = target.to(device)
        
        optim.zero_grad()
        
        pred = model(data)
        
        local_acc = evaluate_batch_accuracy(pred, target)# .cpu()
        accur.append(local_acc)
        
        loss = loss_op(pred, target)
        loss.backward()
        optim.step()
        
        loss_val.append(loss.item())
        
        pbar.set_description("train epoch mean accuracy: %.4f last_acc: %.4f" % (np.mean(accur), local_acc))
        
    return np.mean(accur)

def evaluate_batch_accuracy(y_pred, y_true):
    y_pred = y_pred.cpu().detach().numpy()
    y_true = y_true.cpu().detach().numpy()
    accuracy = 0
    for i in range(len(y_true)):
        index_max = max(range(len(y_pred[i, :])), key=y_pred[i].__getitem__)
        # print(index_max)
        if (index_max == y_true[i]):
            accuracy += 1
    accuracy /= len(y_pred)
    # print('y_pred:', torch.Tensor(y_pred))
    # print('y_true:', torch.Tensor(y_true))
    # print('accura:', accuracy)
    # input('PK')
    return accuracy

    return acc

def valid_epoch(model, loader, device, it_limit=9999):
    accur = []
    #torch.no_grad()
    #model.eval()
    model.to(device)
    
    pbar = tqdm(enumerate(loader),
                total=len(loader),
                position=0,
                leave=False)
        
    for idx, (imgs_batch, labels_batch) in pbar:
        imgs_batch = imgs_batch.to(device)
            
        labels_batch = labels_batch.to(device)

        pred = model(imgs_batch)
        # print('pred', pred)
        # print('pred', labels_batch)
        
        local_acc = evaluate_batch_accuracy(pred, labels_batch)# .cpu()
        # print('acc:', local_acc)
        # input("Press Enter to continue...")
        accur.append(local_acc)
        
        pbar.set_description("valid epoch accuracy: %f" % local_acc)

    return np.mean(accur)

In [ ]:
config = {
    'lr': 0.1,
    'epochs': 9,
    'it_limit': None
}

DEFAULT_MODEL_LOCATION = DATA_DIR / 'resnet18_classifier'

from torchvision import models
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 31)

if os.path.isfile(DEFAULT_MODEL_LOCATION):
    print('[+] Model restored from', DEFAULT_MODEL_LOCATION)
    # model.load_state_dict(torch.load(DEFAULT_MODEL_LOCATION))
    

loss_op = nn.CrossEntropyLoss().cuda()
optim = torch.optim.AdamW(model.parameters(), lr=config['lr'])

model.to(device)

SHOULD_I_TRAIN = True
if SHOULD_I_TRAIN:
    pbar = tqdm(range(config['epochs']),
                total=config['epochs'],
                leave=True)
    
    for epoch in pbar:

        train_res = train_epoch(model, train_loader, loss_op, optim, device, config['it_limit']) # 
        print('train accuracy:', train_res)

        valid_res = valid_epoch(model, train_loader, device, config['it_limit'])
        print('valid accuracy:', valid_res)
        
        #test_res = valid_epoch(model, test_loader, device, config['it_limit'])
        #print('!test:', test_res)
        
        now = datetime.now()
        model_save_name = 'resnet18_classifier_{}_T_ACC{:.4f}_V_ACC{:.4f}'.format(now.strftime("%m.%d_%H.%M"),
                                                                      train_res,
                                                                      valid_res)    
        pbar.set_description("per epoch valid accuracy %f" % valid_res)
        
        torch.save(model.state_dict(), DATA_DIR / model_save_name)
        print('MODEL CHECK CREATED')
        if IN_COLAB:
            shutil.copy2(model_save_name, '/content/drive/MyDrive/')

        torch.save(model.state_dict(), DATA_DIR / 'resnet18_classifier')
        if IN_COLAB:
            shutil.copy2(DATA_DIR / 'resnet18_classifier', '/content/gdrive/MyDrive/')

    

In [ ]:
gt_ = gt[gt["SET"]=='train']
SIGN_SET = set(gt_['SIGN'])

nrows, ncols = 6, 7
fig = plt.figure()

model.to('cpu')

for idx, (img, encoded_label) in enumerate(small_test_loader):
    
    pred = model(img[None, ...])
    
    argmax = np.argmax(pred.detach().numpy())
    model_pred_decoded = le.inverse_transform([argmax])[0]
    model_pred_sign = NUMBER_TO_SIGN[model_pred_decoded]
    # make img from tensor
    img = torch.Tensor.permute(img, [1, 2, 0]).numpy()
    
    # get decoded_label
    decoded_label = le.inverse_transform([encoded_label])[0]
    
    # translate decoded to sign
    sign = NUMBER_TO_SIGN[decoded_label]
    
    ax = fig.add_subplot(nrows, ncols, idx+1)
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), aspect=1)
    ax.set_title('FACT:' + str(sign) + '; PRED:' + str(model_pred_sign))
    
plt.tight_layout()